In [89]:
# Importing some shizzle.

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import time
import requests
import datetime
import sys
import os
import pathlib
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math
import json
import glob

# Defining some import and export locations
location = 'rjsietsma'
read_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Initial_Data_exploration/'
data_expor_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Past_initial_data/'
img_output_dir = '/home/'+location+'/PycharmProjects/dsls_master_thesis/side_scripts/output_img/'

In [90]:
# Some bokeh stuff

from bokeh.plotting import figure, ColumnDataSource, output_notebook, show, save
from bokeh.models import HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool
from bokeh.palettes import inferno

output_notebook

<function bokeh.io.output.output_notebook(resources=None, verbose=False, hide_banner=False, load_timeout=5000, notebook_type='jupyter')>

In [91]:
# Define function to perform Shapiro-Wilk, Kolmogorov-Smirnov, Wilcoxon-/Mann-Whitney U -test, and a pearson correlation test.

def perform_stats(x, y):
    shapiro_x = stats.shapiro(x)[1]
    shapiro_y = stats.shapiro(y)[1]
    if shapiro_x >= 0.05:
        print("X is likely normally distributed.")
    else:
        print("X is likely not normally distributed.")
    if shapiro_y >= 0.05:
        print("Y is likely normally distributed.")
    else:
        print("Y is likely not normally distributed.")
    try:
        p_ks_xy = stats.ks_2samp(x, y)[1]
        print(f"The Kolmogorov-Smirnov test p-value: {p_ks_xy}")
    except Exception:
        print("Kolmogorov-Smirnov could not be performed!")
    try:
        p_wc_xy = stats.wilcoxon(x, y)[1]
        print(f"The Wilcoxon test p-value: {p_wc_xy}")
    except ValueError:
        p_mw_xy = stats.mannwhitneyu(x, y)[1]
        print(f"Wilcoxon could not be performed, \n"
             f"Using Mann-Whitney rank test p-value: {p_mw_xy}")
    except Exception:
        print("Neither Wilcoxon nor Mann-Whitney tests could be performed!")
    try:
        p_pears_xy = stats.pearsonr(x, y)
        print(f"The Pearson correlation: {p_pears_xy[0]},\n"
             f"p-value: {p_pears_xy[1]}")
    except Exception:
        print("Pearson correlation could not be performed!")

# Define function to calculate the Z-scores of given data.

def calc_z_scores(data):
    centered = data - data.mean(axis=0)
    return centered / centered.std(axis=0)

Sprint 28-04-2020 - 19-05-2020 (?)
=====================

- What I want to do:
  1. [Focus on fixing missing CAPICE precomputed scores.](#Fixing-CAPICE-precomputed-scores) __Done__
      - _Note: Find out how the script of Shuang works on a smaller scale, get an example where a CAPICE score is missing._
  2. [Normalize BM_ratio](#Normalize-BM_ratio) for gene length, so $\frac{n_{malignant}}{n_{benign}}$ per 1000 bases in a gene, instead of [$\frac{n_{malign}}{n_{benign}} \times n_{total}$](output_img/bm_ratio.png "BM_ratio").
  3. _(questionable)_ Fix PCA by identifying categorical features _(Questionable because should I still put time and focus in on the input data?)_
      - _Ask Shuang, Joeri and Krista in the PRU 28-04-2020 @ 13:00-14:00_
  4. [See what is required](#Required-for-data-sources) for the [data sources](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit) to be implemented into the input data.
      - _Note: as this is research, I can use sources that only have precomputed scores available, but just to keep in mind that I should mention that in the discussion._

- Notes:
  - If I find myself unfocussed:
      - Win a game of [solitaire](https://play.google.com/store/apps/details?id=com.lemongame.klondike.solitaire&hl=nl) (game is rigged, not every match is win-able).
      - Clean out my closet (it's messy, maybe to clean up the mess inside my head too).
      - Go for a walk around the neighbourhood.

<br/><br/>
<br/><br/>
### Fixing CAPICE precomputed scores

Problem: CAPICE precomputed file of all single nucleotide level variants contains caps.

Solution: Fix the underlying problem that causes these gaps.

Required: 
- Script to make and batch the CAPICE computing __(positive check)__
- Access to GCC cluster __(positive check)__
- Knowledge of job scheduler on GCC cluster __(positive check)__
- Smaller batches of SNV files to run locally  __(positive check)__

Possible cause:
- Incorrect batching on tabix.

31-04-2020 Problem:
- Got it working on Gearshift, but with a problem:
    - AttributeError: Can't get attribute 'DeprecationDict' on <module 'sklearn.utils.deprecation' from '/apps/software/PythonPlus/3.7.4-foss-2018b-v20.02.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py'>
- Which let's me to beleive that the model was trained in a previous version of sklearn.

<br/><br/>

- Update 10:51 : Model compatibility issue. Might have to retrain CAPICE on Gearshift using the latest xgboost, sklearn, scikit-learn, numpy and pandas just to be safe.

- Update 12:01 : Managed to get it working using python3.6 on calculon, essentially downgrading my own script (I only used python3.7 for F-strings)

- Update 15:55 : Found a very weird bug in the chunking. Seems like the script doesn't like to start at 0 with skiprows and 1000 nrows, solution: nrows - 2 when first iteration, then + 1, then start += nrows + 1...

- Update 04-05-2020 : Found out that the first iteration, an overhead of 2 variants was present. Adjusted the start with 2 to compensate. Seemed fine in the first 400000 CADD entries. (no duplicates nor missed entries). __Started full processing__

- Update 04-05-2020 : 
    - Add functionality to continue where left off if it crashes.
    - Make dummy script to check for duplicates or if the files are fully complete.
    
- Update 06-05-2020 : 
    - Functionaility to continue where left off is required. After 40 hours, it's still processing chromosome 1 after 7 million processed variants. 
        - Ideas: write json file with start and stop each iteration once that part is processed and added to the larger files. Should overwrite.
        - As for now, since this idea is not implemented yet, count the number of rows in the already processed files and set that as start.

- Update 11-05-2020:
    - The 4 days have ended, chromosome 1 has not been finished yet.
    - Output file of chromosome 1 has been renamed to a .tsv file and gzipped (saving about 5 gb of storage).
    - Adjustments to the original script have to be made.

#### Version 2.0

- Added gzip output by default
- Currently no function yet to continue where left off.
    - Ideas:
        - for line in gzip.open(file.gz): nlines+=1 (to trace back where was left off)
            - Do not know if this lead to any complications of duplicated entries. Saving the previous results in the RAM memory might solve this to compare to.
    - Then in the future:
        - Save a json of start and batch size each iteration to mark where the file was left of.
- Needs to be added:
    - A save of the last precomputed dataframe and a check if any duplicates are in there. Drop if duplicates exist (and warn in logfile). If no previous precomputed dataframe is present, load in the last _batch_size_ of the output files. List the output files by using glob.glob(x, recursive = True). The order has to be specified aswell (np.array(np.arange(1,23).astype('str').tolist() + ['x', 'y']). __This is potentially very dangerous, this is fully under the assumption that the CADD file is ordered on chromosome.__ _Might be fine for now, since the failsave features will be added now and it's only the run with not enough time that failed._ So in short terms:
        - 
    

<br/><br/>
<br/><br/>
### Normalize BM_ratio

Problem: Larger genes have more benign and malignant samples, because they are larger and thus the chanses of variation on the reference read is higher.

Solution: Make a $n_{malignant}$ and $n_{benign}$ per $x$ bases in a gene, where {$x \in \mathbb{N}\ | x \geq 1 | x \leq 10000$}

<br/><br/>
<br/><br/>
### Required for data sources

Problem: Now that a lot of data sources have been found, annotation and merging has to be performed in order to use it as training data.
- _Additional problem: once a source has been proven beneficial, it has te be implemented to be annotated before capice fits and predicts a new sample._

Solution: Find out, feature by feature if genes that have a low AUC improve *__and that genes that have a high AUC do not perform worse__*

Required:
- Storage 
    - On GCC cluster for precomputed score files.
    - Locally IF file isn't that big or an API can be used.
- Data sources __([positive check](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit))__
- Script to annotate / merge


      

# CTEPG (Catching The Elusive Predictable Genes) 28-04-2020 - 04-05-2020
<br/><br/>

What have I done:

- [Enrichr Genes < 0.6](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=b7a959357214069d1905719220ed3e3b)
- [Enrichr Genes < 0.7](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=7d252a3e1df275859228fd562e964344)
    - For reference: [Enrichr Genes < 0.85](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=60c74741c967c25331419bd2b5e7183e)
- [Finished investigating initial data sources](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit)

<br/><br/>
<br/><br/>

What I'm planning to do this week:

- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores). __Done and not done, script has to have some additonal features__
    - If stuck, do not keep walking into the wall, continue below and contact Shuang and Joeri
- [Run enrichr on increasing cutoffs in AUC (.5, .6, .7 etc.)](#Enrichr-automated-API). __Done__
    - Read into the [Enrichr API documentation](https://amp.pharm.mssm.edu/Enrichr/help#api)
- Mail Tsjerk that a meeting is apreciated. __Done, no response yet__
- Make [suggested plot](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p). __Done__
    - Select from enrichr all the genes involved in (lipid) metabolism and see if this explains all / most of the bad performing genes.
- Make model on bad performing genes and investigate further.
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - Create outline for article.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Fix PCA.
    - Find possible way to annotate genes for co-factors, sub-family etc.
<br/><br/>

# CTEPG (Catching The Elusive Predictable Genes) 04-05-2020 - 11-05-2020
<br/><br/>

What have I done:
- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores).
- [Run enrichr on increasing cutoffs in AUC (.5, .6, .7 etc.)](#Enrichr-automated-API).
- Make [suggested plot](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p).
<br/><br/>

Planned for this week:
- Release precomputed CAPICE script v2.0
- Make model on bad performing genes and investigate further.
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - Create outline for article.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Fix PCA.
    - Find possible way to annotate genes for co-factors, sub-family etc.
 

- Ion transport / transmembrane related genes seems to perform bad. 
- Make plots combined of all sources, but color them by source.

  
    

# Current Tickets:
<br/><br/>

## As student / Robert I want to map requirements for SNV and gene features.

- Content:

Overview of features has now been collected and processed (https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit). In order to use these features, knowledge has to be obtained for the requirements on said features including, but not limited to merging to the trianing dataset and storage.

- Acceptance criterea:
    - Does not take up terrabytes of storage.
        - _If it does, data sources might have to be merged 1 by 1, removing the source after the merge._
    - Can be merged without 3rd party software (tabix, python etc. allowed).
    - Data can easily be downloaded.

<br/><br/>

## As student / Robert I want to find commonalities between bad performing genes in CAPICE

- Content:

The enrichr tool has proven to be quite beneficial to understanding a set of bad performing genes. Further processing of these enrichr results is required in order to fully understand the underlying mechanics and commonalities that these bad performing genes share.

- Acceptance criterea:
    - Enrichr is performed on multiple gene sets of different AUC thresholds (0.5, 0.6, 0.7 etc.).
    - Results of various data sources (kegg, bioplanet, wikipathways etc.) have been obtained in an automated way using the Enrichr API.
    - Results from the Enrichr API have been processed in an understandable plot, an mockup can be found [here](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p).


# Enrichr automated API

In [92]:
enrichr_url = 'http://amp.pharm.mssm.edu/Enrichr/'

In [93]:
# Load in initial data
data = pd.read_csv(read_loc+'optimal_f1_full_ds_v2.csv', header=0)
data.sort_values(by='default_f1', ascending=False)

,gene,default_auc,default_f1,default_recall,default_fpr,default_spec,optimal_c,optimal_auc,optimal_f1,optimal_recall,optimal_fpr,optimal_spec,n_tot,n_benign,n_malign
2775,PRKN,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,11,7,4
1659,BUB1B,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,73,66,7
2102,GALNT3,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,12,10,2
1655,PNPLA1,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,0.8,0.2,0.8,31,26,5
2782,TMEM80,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,C10orf2,0.469697,0.0,0.0,1.0,0.0,0.020,0.469697,0.000000,0.0,1.0,0.0,133,132,1
2330,SPTLC2,0.455157,0.0,0.0,1.0,0.0,0.011,0.750374,0.170213,0.0,1.0,0.0,229,223,6
2615,PLCD1,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,8,7,1
2611,UBE2A,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,3,2,1


In [124]:
data.shape[0]

2807

In [94]:
# Make a dictionary of lists of genes of interest with as key the 0.x threshold

auc_dict = {}
thresholds = np.arange(0.5, 1.01, 0.05).round(decimals=2)
for t in thresholds:
    list_of_genes = data[data['default_auc'] < t]['gene'].tolist()
    auc_dict[t.astype('str')] = '\n'.join(list_of_genes)
    del list_of_genes
del thresholds

# Save it to a file so I don't bombard the Enrichr API with requests
if not os.path.isfile('./user_id_list.json'):
    file = open('./user_id_list.json', 'w')
    file.close()

with open('./user_id_list.json') as json_file:
    try:
        user_list_ids = json.load(json_file)
        json_was_empty = False
    except json.JSONDecodeError:
        user_list_ids = {}
        json_was_empty = True

In [95]:
# Get list ID's for the different AUC's

for threshold in auc_dict.keys():
    if threshold not in user_list_ids.keys():
        description = f'CAPICE performance of genes with an AUC lower than {threshold}'
        payload = {
            'list': (None, auc_dict[threshold]),
            'description': (None, description)
        }
        response = requests.post(enrichr_url + '/addList', files=payload)
        if not response.ok:
            print(f'Something went wrong with {threshold}!')
        else:
            enrichr_addlist_response = json.loads(response.text)
            user_list_ids[threshold] = enrichr_addlist_response['userListId']

# Write it to a file in case I didn't have any.
if json_was_empty:
    with open('./user_id_list.json', 'w') as json_file:
        json.dump(user_list_ids, json_file)

In [96]:
# Now to get the enrichment results.

enrichr_sources_oi = ['BioPlanet_2019', 'WikiPathways_2019_Human', 'KEGG_2019_Human', 'Elsevier_Pathway_Collection',
                     'GO_Biological_Process_2018',
                     'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET',
                     'Human_Gene_Atlas', 'ARCHS4_Tissues']


In [97]:
# Now get the enrichr results.
def get_enrichr_results(enrichr_sources, number_of_hits):
    if len(enrichr_sources) == 0:
        print('You should give some sources for me to get data from.')
    else:

        # Prepare output dataframes for each source
        output_dir = '/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/'
        
        for iteration, source in enumerate(enrichr_sources):
            print(f'Working on source: {source}')
            if iteration == len(enrichr_sources):
                sources_left = []
            else:
                sources_left = enrichr_sources[iteration+1::]
            print(f'Sources left: {sources_left}')
            # Prepare the output dict
            current_source_df = pd.DataFrame(columns=['AUC',
                                                      'Rank',
                                                      'Term_Name',
                                                      'Overlapping_Genes',
                                                      'Adjusted_P-value',
                                                      'Source'])
            for (threshold, user_id) in zip(user_list_ids.keys(),user_list_ids.values()):
                print(f'\tCurrently working on threshold: {threshold}')
                req = enrichr_url + f'enrich?userListId={user_id}&backgroundType={source}'
                response = requests.get(req)
                if not response.ok:
                    print(f'Userid {user_id} (threshold: {threshold})'
                          f'received error on: {source}!')
                else:
                    enrichr_response_data = json.loads(response.text)
                    content = enrichr_response_data[source]
                    auc = threshold
                    content_df = pd.DataFrame(columns=['AUC', 'Rank', 'Term_Name',
                                                       'Combined_score',
                                                       'Overlapping_Genes',
                                                       'Adjusted_P-value',
                                                       'Source'])
                    for c in content:
                        rank = c[0]
                        term_name = c[1]
                        combined_score = c[4]
                        overlapping_genes = ', '.join(c[5])
                        a_p_value = c[6]
                        local_df = pd.DataFrame({'AUC':auc, 'Rank':rank,
                                                 'Term_Name': term_name,
                                                 'Combined_score': combined_score,
                                                 'Overlapping_Genes': overlapping_genes,
                                                 'Adjusted_P-value': a_p_value,
                                                'Source': source}, index=[0])
                        content_df = content_df.append(local_df, ignore_index=True)
                    content_df = content_df.sort_values(by=['Adjusted_P-value',
                                                            'Combined_score'],
                                                        ascending=[True, False])
                    max_rows = 0
                    if content_df.shape[0] < number_of_hits:
                        max_rows = content_df.shape[0] - 1
                    else:
                        max_rows = number_of_hits
                    out_df = content_df[:max_rows]
                    out_df.drop('Combined_score', axis=1, inplace=True)
                    current_source_df = current_source_df.append(out_df, ignore_index=True)
            output_filename = output_dir + source + '.csv'
            current_source_df.to_csv(output_filename, index=False, sep=',')
            print(f'\tSource {source} has been exported to csv (\n\t\t{output_filename}\n\t)')

In [98]:
output_source_link = {}
for source in enrichr_sources_oi:
    output_filename = source + '.csv'
    output_source_link[source] = output_filename

In [99]:
glob_files_present = glob.glob('./EnrichrAPIResults/*.csv')
list_of_files_present = []
if len(glob_files_present) > 0:
    for file in glob_files_present:
        list_of_files_present.append(file.split('/')[-1])

In [100]:
sources_required = []
for source, output_filename in output_source_link.items():
    if output_filename not in list_of_files_present:
        sources_required.append(source)

In [101]:
sources_required

['BioPlanet_2019',
 'WikiPathways_2019_Human',
 'KEGG_2019_Human',
 'Elsevier_Pathway_Collection',
 'GO_Biological_Process_2018',
 'PheWeb_2019',
 'ClinVar_2019',
 'GWAS_Catalog_2019',
 'DisGeNET',
 'Human_Gene_Atlas',
 'ARCHS4_Tissues']

In [102]:
get_enrichr_results(sources_required, 20)

Working on source: BioPlanet_2019
Sources left: ['WikiPathways_2019_Human', 'KEGG_2019_Human', 'Elsevier_Pathway_Collection', 'GO_Biological_Process_2018', 'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET', 'Human_Gene_Atlas', 'ARCHS4_Tissues']
	Currently working on threshold: 0.5


/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently working on threshold: 0.7
	Currently working on threshold: 0.75
	Currently working on threshold: 0.8
	Currently working on threshold: 0.85
	Currently working on threshold: 0.9
	Currently working on threshold: 0.95
	Currently working on threshold: 1.0
	Source BioPlanet_2019 has been exported to csv (
		/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/BioPlanet_2019.csv
	)
Working on source: WikiPathways_2019_Human
Sources left: ['KEGG_2019_Human', 'Elsevier_Pathway_Collection', 'GO_Biological_Process_2018', 'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET', 'Human_Gene_Atlas', 'ARCHS4_Tissues']
	Currently working on threshold: 0.5
	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently working on threshold: 0.7
	Currently working on threshold: 0.75
	Cur

In [103]:
enrichr_data_loc = './EnrichrAPIResults/'

enrichr_dataset = {}

for source in enrichr_sources_oi:
    dataset = enrichr_data_loc + source + '.csv'
    enrichr_dataset[source] = pd.read_csv(dataset)

In [104]:
enrichr_dataset['BioPlanet_2019'].head()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019


In [117]:
def plot_results(source, n=5, print_messages=False):
    source_name = source['Source'][0]
    term_values = source['Term_Name'].value_counts()
    if print_messages:
        print(f'The most occuring term names in {source_name} is: \n{term_values[:n]}')
    plot_source = pd.DataFrame(columns=source.columns)
    for auc in source['AUC'].unique():
        subsource = source[source['AUC'] == auc][:n]
        plot_source = plot_source.append(subsource)
    category = 'Term_Name'
    category_items = plot_source[category].unique()
    palette = inferno(len(category_items) + 1)
    colormap = dict(zip(category_items, palette))
    plot_source['color'] = plot_source[category].map(colormap)
    
    title = f'Term names from {source_name}, n={n}'
    source_bokeh = ColumnDataSource(plot_source)
    hover = HoverTool(tooltips=[('Term_name', '@Term_Name'), ('Genes', '@Overlapping_Genes')])
    tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), SaveTool()]
    
    p = figure(tools=tools, title=title, plot_width=2000, plot_height=1200,toolbar_location='right', toolbar_sticky=False,)
    if n > 5:
        p.scatter(x='AUC',y='Adjusted_P-value', source=source_bokeh, size=10, color='color')
    else:
        p.scatter(x='AUC',y='Adjusted_P-value', source=source_bokeh, size=10, color='color', legend=category)
    p.xaxis.axis_label = 'AUC'
    p.yaxis.axis_label = 'Adjusted P-value'
    return p
    

In [114]:
glob_pictures_present = glob.glob('./EnrichrAPIResults/*.html')
list_of_pictures_present = []
if len(glob_pictures_present) > 0:
    for file in glob_pictures_present:
        list_of_pictures_present.append(file)
output_picture_link = {}
for source in enrichr_sources_oi:
    output_filename = './EnrichrAPIResults/' + source + '.html'
    output_picture_link[source] = output_filename
pictures_required = []
for source, output_picture in output_picture_link.items():
    if output_picture not in list_of_pictures_present:
        pictures_required.append(source)

In [115]:
output_picture_link

{'BioPlanet_2019': './EnrichrAPIResults/BioPlanet_2019.html',
 'WikiPathways_2019_Human': './EnrichrAPIResults/WikiPathways_2019_Human.html',
 'KEGG_2019_Human': './EnrichrAPIResults/KEGG_2019_Human.html',
 'Elsevier_Pathway_Collection': './EnrichrAPIResults/Elsevier_Pathway_Collection.html',
 'GO_Biological_Process_2018': './EnrichrAPIResults/GO_Biological_Process_2018.html',
 'PheWeb_2019': './EnrichrAPIResults/PheWeb_2019.html',
 'ClinVar_2019': './EnrichrAPIResults/ClinVar_2019.html',
 'GWAS_Catalog_2019': './EnrichrAPIResults/GWAS_Catalog_2019.html',
 'DisGeNET': './EnrichrAPIResults/DisGeNET.html',
 'Human_Gene_Atlas': './EnrichrAPIResults/Human_Gene_Atlas.html',
 'ARCHS4_Tissues': './EnrichrAPIResults/ARCHS4_Tissues.html'}

In [119]:
if len(pictures_required) > 0:
    for plot_req in pictures_required:
        plot = plot_results(enrichr_dataset[plot_req], 5)
        save(plot, output_picture_link[plot_req])

/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/bokeh/io/saving.py:138: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
